In [1]:
import pickle
import numpy as np
from dataset import Dataset
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, SGDRegressor, BayesianRidge

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

dataset = Dataset(sensor_data, labels, interp_funcs)

In [3]:
rs = 42
results = {
    "mat_0": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []},
    "mat_1": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []}
}
r2_dict = {
    "LR": [],
    "GBR": [],
    "EN": [],
    "SGDR": [],
    "SVR": [],
    "BR": []
}
mae_dict = {
    "LR": [],
    "GBR": [],
    "EN": [],
    "SGDR": [],
    "SVR": [],
    "BR": []
}

for matrix in range(2):
    for s in range(0, 8, 2):
        s_l, s_r = s, s+1
        print(f"mat {matrix}, s: {s_l} - {s_r}")
        X, y, _, targets = dataset.get_sensor_pair_cls(matrix,
                                                       (s_l, s_r),
                                                       num_samples=100,
                                                       as_log=True,
                                                       as_mean=False)
        X, y = dataset.clean_up_regression_data(X, targets)
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            test_size=0.2,
                                                            random_state=rs)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = LinearRegression()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        reg = "LR"
        score_r2 = np.round(r2_score(y_test, y_pred), 3)
        score_mae = np.round(mean_absolute_error(y_test, y_pred), 3)
        r2_dict[reg].append(score_r2)
        mae_dict[reg].append(score_mae)

        model = GradientBoostingRegressor()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        reg = "GBR"
        score_r2 = np.round(r2_score(y_test, y_pred), 3)
        score_mae = np.round(mean_absolute_error(y_test, y_pred), 3)
        r2_dict[reg].append(score_r2)
        mae_dict[reg].append(score_mae)

        model = ElasticNet()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        reg = "EN"
        score_r2 = np.round(r2_score(y_test, y_pred), 3)
        score_mae = np.round(mean_absolute_error(y_test, y_pred), 3)
        r2_dict[reg].append(score_r2)
        mae_dict[reg].append(score_mae)

        model = SGDRegressor()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        reg = "SGDR"
        score_r2 = np.round(r2_score(y_test, y_pred), 3)
        score_mae = np.round(mean_absolute_error(y_test, y_pred), 3)
        r2_dict[reg].append(score_r2)
        mae_dict[reg].append(score_mae)

        model = SVR()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        reg = "SVR"
        score_r2 = np.round(r2_score(y_test, y_pred), 3)
        score_mae = np.round(mean_absolute_error(y_test, y_pred), 3)
        r2_dict[reg].append(score_r2)
        mae_dict[reg].append(score_mae)

        model = BayesianRidge()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        reg = "BR"
        score_r2 = np.round(r2_score(y_test, y_pred), 3)
        score_mae = np.round(mean_absolute_error(y_test, y_pred), 3)
        r2_dict[reg].append(score_r2)
        mae_dict[reg].append(score_mae)

mat 0, s: 0 - 1
mat 0, s: 2 - 3
mat 0, s: 4 - 5
mat 0, s: 6 - 7
mat 1, s: 0 - 1
mat 1, s: 2 - 3
mat 1, s: 4 - 5
mat 1, s: 6 - 7


In [4]:
import pandas as pd
df = pd.DataFrame(r2_dict)
df.to_excel("regressor_scores_r2.xlsx")
df = pd.DataFrame(mae_dict)
df.to_excel("regressor_scores_mae.xlsx")